In [79]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
from pandas import read_csv
from sklearn.utils import shuffle
from datetime import datetime

In [80]:
df_data = pd.read_pickle('CleanedDataWithIds.pkl') 
df_data = shuffle(df_data)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [81]:
df_data.head()

,elementId,timeStamp,type,siteName,date,clientX,clientY,session,height,width,browser,key,sitename
9309,inputDays,81312.60000000894,mouseout,1,2022-10-19T06:33:14.147Z,869,443,495,1336,1720,NaN,NaN,NaN
8448,inputRiver,224915.1000000001,mouseout,1,2022-10-19T06:25:56.771Z,882,609,937,1304,2117,NaN,NaN,NaN
35284,inputHouse,10768.09999999404,keydown,0,2022-10-25T11:19:02.128Z,NaN,NaN,652,914,1680,NaN,Backspace,NaN
6331,inputRiver,487704.60000002384,mouseover,1,2022-10-18T14:17:24.067Z,946,582,243,1328,2560,NaN,NaN,NaN
35928,inputDays,21353,blur,1,2022-10-27T21:36:03.985Z,NaN,NaN,874,868,1544,Firefox,NaN,NaN


# Hier kommt DataFrame sortiert nach Session

In [82]:
df_replaced_together = df_data.sort_values(by=['session','date'], ascending=[False, True])

In [83]:
df_replaced_together['real_date'] = df_replaced_together['date'].astype('datetime64[ns]')
#df_replaced_together['real_date'] = (df_replaced_together['real_date'] - df_replaced_together['real_date'].min())  / np.timedelta64(1,'D')
df_replaced_together = df_replaced_together.drop(['date', 'clientX', 'clientY', 'sitename', 'key', 'timeStamp'], axis=1)
df_replaced_together = df_replaced_together.loc[df_replaced_together["type"] !="scroll"]

In [84]:
#get all new id numbers 
sessionId = df_replaced_together['session']
df_replaced_together['is_duplicated'] = df_replaced_together.duplicated('session')
#86 ids auf Seite Einfach
numbers1 = set(sessionId)
numbers = list(numbers1)

print(numbers)

[512, 518, 519, 520, 538, 555, 588, 589, 591, 104, 622, 128, 134, 647, 137, 650, 651, 652, 142, 656, 149, 662, 683, 176, 689, 693, 186, 188, 711, 712, 199, 715, 717, 728, 218, 221, 222, 737, 748, 237, 240, 754, 243, 244, 242, 246, 759, 760, 247, 762, 253, 766, 255, 259, 772, 268, 786, 787, 277, 280, 290, 298, 813, 820, 309, 829, 837, 838, 842, 330, 846, 338, 853, 342, 856, 347, 349, 350, 353, 871, 874, 363, 369, 372, 890, 384, 898, 901, 904, 393, 395, 914, 917, 410, 925, 926, 927, 416, 929, 421, 937, 427, 952, 441, 955, 444, 958, 962, 964, 966, 455, 456, 454, 975, 464, 465, 463, 981, 474, 475, 476, 990, 991, 480, 995, 997, 487, 495, 503, 504, 505, 508]


In [85]:
hey = pd.DataFrame(numbers, columns=["session"])
arr1 = []
arr2 = []
arr3 = []
arr4 = []
arr5 = []
arr6 = []
arr7 = []
arr8 = []
arrDate = []
arrHeight = []
arrWidth = []
arrSiteName = []

for i in range(len(numbers)):
    new = df_replaced_together.loc[df_replaced_together['session'] == numbers[i]]
    #get first and last entry of new dataframe
    #first = new['real_date'].iloc[0]
    date = (new['real_date'].iloc[-1] - (new['real_date'].iloc[0])).total_seconds()
    arrDate.append(date)
    height = new['height'].iloc[0]
    arrHeight.append(height)
    width = new['width'].iloc[0]
    arrWidth.append(width)
    siteName = new['siteName'].iloc[0]
    arrSiteName.append(siteName)
    #print(new)
    #print(date)
    # last = new.iloc[-1]
    new2 = (len(new.loc[new['type']=='mouseover']))
    arr2.append(new2)
    new3 = (len(new.loc[new['type'] =='mouseout']))
    arr3.append(new3)
    new4 = (len(new.loc[new['type'] =='click']))
    arr4.append(new4)
    new5 = (len(new.loc[new['type'] =='blur']))
    arr5.append(new5)
    new6 = (len(new.loc[new['type'] =='focus']))
    arr6.append(new6)
    new7 = (len(new.loc[new['type'] =='keydown']))
    arr7.append(new7)

hey['count_mouseover'] = arr2
hey['count_mouseout'] = arr3
hey['count_click'] = arr4
hey['count_blur'] = arr5
hey['count_focus'] = arr6
hey['count_keydown'] = arr7
hey['duration'] = arrDate
hey['height'] = arrHeight
hey['width'] = arrWidth
hey['siteName'] = arrSiteName

# for i in range(len(types)):
#     hey['type_' + str(types[i])] = arr



#hey['duration'] = df_replaced_together['date']


In [86]:
hey = hey.loc[hey['duration'] < 2000]

In [87]:
hey.head()

,session,count_mouseover,count_mouseout,count_click,count_blur,count_focus,count_keydown,duration,height,width,siteName
0,512,7,7,0,7,7,4,18.157,664,390,0
1,518,70,70,25,31,32,13,459.177,667,1440,1
2,519,6,6,0,7,7,0,18.090,775,412,0
3,520,39,39,3,14,13,29,232.459,1217,1440,1
4,538,8,8,4,17,15,20,213.904,664,390,1


In [88]:
hey.to_pickle('SortedBySessions.pkl')